In [1]:
import torch
import numpy as np
import segmentation_models_pytorch as smp

import torch.nn as nn
from torchmetrics.functional import accuracy

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger

from torchlightning_module import torch_lightning_DataModule
from dataset_module import DatasetModule

#torch.set_float32_matmul_precision('medium')

###
class SemanticSegmentationModel(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        for key in hparams.keys():
             self.hparams[key]=hparams[key]

        self.loss_fn = nn.CrossEntropyLoss()
        self.save_hyperparameters()
        
        self.model = smp.FPN( #modifiy in-channels and output. 
           encoder_name=self.hparams['encoder_name'], 
           classes=self.hparams['classes'], 
           activation=self.hparams['activation'], 
           encoder_weights=self.hparams['encoder_weights'],
            in_channels=self.hparams['in_channels'],

              )

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self.model(images)
        labels = labels.squeeze(1).long() 
        loss = self.loss_fn(outputs, labels)
        
        
        #outputs : output of model > logits /// labels > y ///images > x 
        self.log('train_loss',loss, on_step=False, on_epoch=True)
        return loss
    
    
#     def training_epoch_end(self, outputs):
#         avg_loss = torch.stack([x["loss"] for x in outputs]).mean()
#         self.log("avg_train_loss", avg_loss)
#         wandb.log({"avg_train_loss": avg_loss})


    def validation_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self.model(images)
        labels = labels.squeeze(1).long()
        #print(f'{labels.shape} is shape of label in the val step method')
        #print(f'{outputs.shape} is shape of outputs in the val step method')

        loss = self.loss_fn(outputs, labels)
        # Log loss and accuracy
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        
        return {'val_loss': loss}

#     def validation_epoch_end(self, outputs):
#         avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
#         logs = {'val_loss': avg_loss}
#         return {'val_loss': avg_loss, 'log': logs}

        # def validation_epoch_end(self, outputs):
        #     avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        #     logs = {'val_loss': avg_loss}
        #     self.log('val_loss', avg_loss)



    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

    
# def lr_schedule(step):
#    lr = 0.001
#    if step < 10:
#        return lr
#    elif step < 20:
#        return lr / 2
#    else:
#        return lr / 4

#lr_scheduler = pl.callbacks.LearningRateScheduler(lr_schedule)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    mode='min'
)


# Define your hyperparameters
hparams = {
    'encoder_name': 'resnet34', #resnet50
    'classes': 19,
    'activation': 'softmax',
    'encoder_weights': 'imagenet', #imagenet
    'lr': 0.001,#  
    'in_channels':5,

}

# Initialize your model
model = SemanticSegmentationModel(hparams)
data=torch_lightning_DataModule()

wandb_logger = WandbLogger(project='phase2_semantic_segmentation_initial_run')

trainer = pl.Trainer(max_epochs=10,
                     accelerator='gpu',
                     callbacks=[early_stopping],
                     logger=wandb_logger
                    ) #lr_scheduler

trainer.fit(model,data)

wandb: Currently logged in as: markalsa. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 1 - CUDA_VISIBLE_DEVICES: [0,1]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0

Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:217: UserWarning: strategy=ddp_spawn and num_workers=0 may result in data loading bottlenecks. Consider setting num_workers>0 and persistent_workers=True
  rank_zero_warn(


ProcessExitedException: process 1 terminated with signal SIGSEGV

In [2]:
import multiprocessing

num_workers = multiprocessing.cpu_count() - 1


In [3]:
num_workers

31